In [1]:
import numpy as np
import tensorflow as tf
from random import randint
import scipy as scipy
import scipy.interpolate as interpolate
import math
from random import *
from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer
from scipy.stats import logistic
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt2
import pandas as pd

import tflib as lib
import tflib.ops.linear
import tflib.ops.conv2d
import tflib.ops.batchnorm
import tflib.ops.deconv2d
import tflib.save_images
import tflib.plot


Lambda = 10.0
BATCH_SIZE = 30
BATCH_SIZE_2 = 100
dimensionality = 13
num_epochs = 100
num_iters = 1000
k=2
max_grad_norm = 1000
gamma = 1.0

# main parameters

rho = 0.5
C=10.0

# additional parameters
sigma=0.8

number_of_neurons_regr = 100
number_of_points = 303-33
number_of_neurons = number_of_points

/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
heart = pd.read_csv('/home/rust/Desktop/SDR/heart.csv')
X = heart.iloc[:,:-1].values
y = heart.iloc[:,13].values
print(len(y))
data = []
target = []
rr = np.random.permutation(303)
for x in rr:
    data.append(X[x])
    target.append(y[x])
# training data
_train_x = np.array(data)
_train_y = np.array(target)
print(_train_x.shape)
print(_train_y.shape)

303
(303, 13)
(303,)


In [3]:
new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
Dataplace = tf.placeholder(tf.float32, shape=(dimensionality, number_of_points))
Data = tf.placeholder(tf.float32, shape=(BATCH_SIZE, dimensionality))
outputs = tf.placeholder(tf.float32, shape=(BATCH_SIZE,1)) 

tf_data_x = gamma*tf.random_normal([BATCH_SIZE, dimensionality]) # аргументы функции
tf_data_y = tf.reduce_mean(tf.math.cos(tf.matmul(tf_data_x, Dataplace)), axis=1) # значения функции

tf_data_w = tf.placeholder(tf.float32, shape=(number_of_neurons,1))
tf_data_B = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons))
tf_data_P = tf.placeholder(tf.float32, shape=(dimensionality, dimensionality))

tf_w_regr = tf.placeholder(tf.float32, shape=(number_of_neurons_regr,1))
tf_B_regr = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons_regr))
tf_bias_regr = tf.placeholder(tf.float32, shape=(number_of_neurons_regr))


tf_data_second = gamma*tf.random_normal([BATCH_SIZE_2, dimensionality])
Lbd = tf.placeholder(tf.float32, shape=[], name="lambda")

# characteristic function parameters
w = tf.Variable(tf.random_normal([number_of_neurons,1], stddev=0.0), name="neuron_weights")
B = tf.Variable(initial_value=Dataplace, name="weights")
# regression function parameters
w_regr = tf.Variable(tf.random_normal([number_of_neurons_regr,1], stddev=0.35), name="neuron_weights")
B_regr = tf.Variable(tf.random_normal([dimensionality,number_of_neurons_regr], stddev=0.35), name="weights")
bias_regr = tf.Variable(tf.random_normal([number_of_neurons_regr], stddev=0.0), name="biases")

prediction = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_x, B)), tf.nn.sigmoid(w))
penalty = tf.square((2/number_of_neurons)*tf.reduce_sum(tf.nn.sigmoid(w))-1.0)

out_loss = (1-rho)*tf.reduce_mean(tf.square(prediction - tf_data_y)) + C*penalty

prediction2 = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_second, B)), tf.nn.sigmoid(w))
prediction_regr2 = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(tf_data_second, B_regr), bias_regr)), w_regr)

grad_psi = tf.reshape(tf.gradients(prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])
grad_psi_regr = tf.reshape(tf.gradients(prediction_regr2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

tf_prediction2 = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_second, tf_data_B)), tf.nn.sigmoid(tf_data_w))
tf_prediction_regr2 = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(tf_data_second, tf_B_regr), tf_bias_regr)), tf_w_regr)

tf_data_grad_psi = tf.reshape(tf.gradients(tf_prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])
tf_data_grad_psi_regr = tf.reshape(tf.gradients(tf_prediction_regr2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

old_part = tf.matmul(tf_data_grad_psi, tf_data_P)
old_part_regr = tf.matmul(tf_data_grad_psi_regr, tf_data_P)

loss = out_loss + Lbd*(1-rho)*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(grad_psi, old_part)), axis=1)) + Lbd*rho*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(grad_psi_regr, old_part_regr)), axis=1))

#offset = tf.random.uniform(shape=[], minval=0, maxval=12, dtype=tf.int32)*BATCH_SIZE
x_plus_error = Data+sigma*tf.random_normal([BATCH_SIZE, dimensionality])
regression = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(x_plus_error, B_regr), bias_regr)), w_regr)
sdr = -tf.reduce_mean(tf.multiply(regression, outputs)) + tf.reduce_mean(tf.math.log(1.0+tf.math.exp(regression)))

loss_hybrid = loss + rho*sdr

In [4]:
def euclidean_distance(pnt1, pnt2):
    '''Finds the distance between 2 points: pnt1, pnt2'''
    # element-wise computations are automatically handled by numpy
    return sum((pnt1 - pnt2) ** 2)

from collections import defaultdict

def find_majority(labels):
    '''Finds the majority class/label out of the given labels'''
    # defaultdict(type) is to automatically add new keys without throwing error.
    counter = defaultdict(int)
    for label in labels:
        counter[label] += 1

    # Finding the majority class.
    majority_count = max(counter.values())
    for key, value in counter.items():
        if value == majority_count:
            return key

def new_predict(k, train_pnts, train_labels, test_pnts):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_pnts, pnt), label)
                    for (pnt, label) in zip(train_pnts, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return find_majority(k_labels)

def new_predict_regr(k, train_pnts, train_labels, test_pnts):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_pnts, pnt), label)
                    for (pnt, label) in zip(train_pnts, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return np.mean(k_labels)

In [5]:
target = tf.train.AdamOptimizer(learning_rate=3e-5, beta1=0.5, beta2=0.9).minimize(loss_hybrid)

In [6]:
def sigmoid(x):
  return 1. /(1+np.exp(-x))

In [ ]:
scores = []
scores3 = []
for part in range(9):
    print("Current part is %d\n" % part)
    train_x = np.concatenate((_train_x[0:(part*33)], _train_x[(part+1)*33:303]), axis=0)
    train_y = np.concatenate((_train_y[0:(part*33)], _train_y[(part+1)*33:303]), axis=0)
    test_x = _train_x[part*33:(part+1)*33]
    test_y = _train_y[part*33:(part+1)*33]
    scaler = preprocessing.StandardScaler().fit(train_x)
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)
    transpose_train_x = np.transpose(train_x)

    cur_w = np.random.normal(0, 0.35, (number_of_neurons,1))
    cur_B =  np.random.normal(0, 0.35, (dimensionality, number_of_neurons))
    cur_biases = np.zeros((number_of_neurons))
    cur_P = np.zeros((dimensionality, dimensionality)) 
    cur_w_regr = np.random.normal(0, 0.35, (number_of_neurons_regr,1))
    cur_B_regr = np.random.normal(0, 0.35, (dimensionality, number_of_neurons_regr))
    cur_bias_regr = np.random.normal(0, 0.35, (number_of_neurons_regr))
    
    O = np.zeros((dimensionality, k)) 
    cur_iter = 0
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init, feed_dict={Dataplace: transpose_train_x})
    
    for epoch in range(num_epochs):
        print("Epoch %d" %(epoch))
        for iteration in range(num_iters):
            offset = (cur_iter % 9)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            sess.run(target, feed_dict={Data: sample_x, outputs:sample_y, Lbd: Lambda, Dataplace: transpose_train_x,
                                        tf_data_w: cur_w,
                                        tf_data_B: cur_B,
                                        tf_data_P: cur_P,
                                        tf_w_regr: cur_w_regr,
                                        tf_B_regr: cur_B_regr,
                                        tf_bias_regr: cur_bias_regr})
            cur_iter = cur_iter+1
        reses = []
        for i in range(100):
            offset = (cur_iter % 9)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            res = sess.run([loss_hybrid], feed_dict={Data: sample_x, outputs:sample_y, Lbd: Lambda, Dataplace: transpose_train_x, 
                                        tf_data_w: cur_w,
                                        tf_data_B: cur_B,
                                        tf_data_P: cur_P,
                                        tf_w_regr: cur_w_regr,
                                        tf_B_regr: cur_B_regr,
                                        tf_bias_regr: cur_bias_regr})
            cur_iter = cur_iter+1
            reses.append(res)
        print ("Iter %d: loss: %.4f\n" %(cur_iter, np.mean(np.array(reses))))
        lib.plot.plot('train cost', np.mean(np.array(reses)))

        cur_w, cur_B, cur_w_regr, cur_B_regr, cur_bias_regr = sess.run([w, B, w_regr, B_regr, bias_regr])

        third_grad_psi = np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))
        third_grad_psi_regr = np.reshape(sess.run([grad_psi_regr]), (BATCH_SIZE_2, dimensionality))
        for r in range(1000):
            sess.run([tf_data_second])
            np.concatenate((third_grad_psi, np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))), axis=0) 
            np.concatenate((third_grad_psi_regr, np.reshape(sess.run([grad_psi_regr]), (BATCH_SIZE_2, dimensionality))), axis=0)

        M = (1-rho)*np.matmul(np.transpose(third_grad_psi), third_grad_psi)+\
            rho*np.matmul(np.transpose(third_grad_psi_regr), third_grad_psi_regr)
        u, s, vh = np.linalg.svd(M, full_matrices=True)
        O = u[:,0:k:1]
        print(s)
        cur_P = np.matmul(O, np.transpose(O))
        tvr = 1-np.sum(np.multiply(s[0:k],s[0:k]))/np.sum(np.multiply(s,s))
        lib.plot.plot('total variance to retain', tvr)
        lib.plot.tick()
        lib.plot.flush()
    reduced_train_x = np.matmul(train_x, O)
    reduced_test_x = np.matmul(test_x, O)
    clf = LogisticRegression(random_state=0).fit(reduced_train_x, train_y)
    score = clf.score(reduced_test_x, test_y)
    scores.append(score)
    print ("Part %d: rate on test %.4f\n" %(part, score))
    test_rgb = []
    for h in test_y:
        if h == 1:
            test_rgb.append('r')
        else:
            test_rgb.append('b')
    plt2.close()
    plt2.scatter(np.transpose(reduced_test_x)[0], np.transpose(reduced_test_x)[1], alpha=0.2, c=np.array(test_rgb))
    plt2.xlabel("1st component")
    plt2.ylabel("2nd component")
    plt2.show()
    sess.close()
    print(cur_P)
    
    i = 0
    total_correct = 0
    for test_image in reduced_test_x:
        pred = new_predict(10, reduced_train_x, train_y, test_image)
        if pred == test_y[i]:
            total_correct += 1
        score3 = (total_correct / (i+1)) * 100            
        i += 1
    print('acc:', str(round(score3, 2))+'%')
    scores3.append(score3)

print ("Average rate on test %.4f\n" %(np.mean(np.array(scores))))
print ("Average 10-NN acc on test %.4f\n" %(np.mean(np.array(scores3))))

Current part is 0

Epoch 0
Iter 1100: loss: 10.7542

[41.827614  16.936243   9.577354   8.666677   7.6756167  6.6718674
  5.159369   3.9996836  2.9534886  2.2777178  1.826985   1.5583429
  1.4812065]
iter 1	train cost	10.754154205322266	total variance to retain	0.14117854833602905
Epoch 1
Iter 2200: loss: 4.6002

[26.505621  13.711776   7.4149857  6.6231456  4.439984   3.7930393
  3.5690017  2.6627913  2.1086833  1.869579   1.3752363  1.3081744
  1.094559 ]
iter 2	train cost	4.60021448135376	total variance to retain	0.15672999620437622
Epoch 2
Iter 3300: loss: 3.2562

[30.805477  12.119826   5.2561827  4.126224   3.373397   2.9705784
  2.5360003  2.0524929  1.634194   1.3071406  1.1560993  1.0442059
  0.6964951]
iter 3	train cost	3.2561721801757812	total variance to retain	0.07024288177490234
Epoch 3
Iter 4400: loss: 2.3257

[27.20281    10.908893    4.071526    3.249227    2.5737846   2.071537
  1.6959594   1.4078045   0.9774981   0.8524917   0.8177818   0.73552275
  0.6327912 ]
iter 

Iter 34100: loss: 0.3841

[20.211464    3.571383    0.19473925  0.15468691  0.12071341  0.11109167
  0.1025493   0.09071996  0.07454979  0.05945069  0.04773057  0.03330637
  0.02924774]
iter 31	train cost	0.38406360149383545	total variance to retain	0.0002868175506591797
Epoch 31
Iter 35200: loss: 0.3794

[20.52051     4.787543    0.19989178  0.1711772   0.13490643  0.10351316
  0.09835576  0.0851928   0.06695773  0.06341775  0.04834347  0.03985502
  0.03917544]
iter 32	train cost	0.3793615400791168	total variance to retain	0.0002905726432800293
Epoch 32
Iter 36300: loss: 0.3769

[18.748327    4.6935806   0.14935635  0.11449033  0.10421561  0.08538733
  0.07616009  0.07271955  0.05883348  0.04998924  0.04655363  0.04411939
  0.03200497]
iter 33	train cost	0.3769434690475464	total variance to retain	0.00020271539688110352
Epoch 33
Iter 37400: loss: 0.3757

[20.016495    4.26997     0.19084428  0.13482678  0.12761402  0.10290785
  0.08275277  0.0795683   0.06737026  0.06667872  0.0514485

Iter 66000: loss: 0.3204

[2.98679733e+01 3.91836405e+00 1.39315397e-01 1.07797444e-01
 8.02638158e-02 7.75370523e-02 6.44958615e-02 5.83101399e-02
 5.21612614e-02 4.48609181e-02 3.88478637e-02 3.82175744e-02
 2.68350709e-02]
iter 60	train cost	0.32036668062210083	total variance to retain	6.556510925292969e-05
Epoch 60
Iter 67100: loss: 0.3154

[2.9724022e+01 3.9391670e+00 1.1143416e-01 8.5991234e-02 7.7949770e-02
 6.6767298e-02 6.6271193e-02 5.3362343e-02 4.9302958e-02 4.1550096e-02
 3.6755398e-02 2.5564456e-02 2.1215176e-02]
iter 61	train cost	0.31538164615631104	total variance to retain	4.9173831939697266e-05
Epoch 61
Iter 68200: loss: 0.3160

[3.05096912e+01 4.20830727e+00 1.05439976e-01 8.93008709e-02
 7.85195157e-02 5.63405901e-02 5.11824302e-02 4.91598584e-02
 4.80686910e-02 3.97210233e-02 2.92215105e-02 2.57232562e-02
 2.41441820e-02]
iter 62	train cost	0.31600141525268555	total variance to retain	4.172325134277344e-05
Epoch 62
Iter 69300: loss: 0.3166

[3.0431829e+01 4.7168026

Iter 95700: loss: 0.2880

[3.7843834e+01 4.5017910e+00 9.9407546e-02 7.1558684e-02 5.5395581e-02
 5.0702270e-02 4.1920528e-02 3.8633756e-02 3.4843329e-02 2.8262980e-02
 2.4408275e-02 2.4245633e-02 1.8116925e-02]
iter 87	train cost	0.2880011796951294	total variance to retain	1.901388168334961e-05
Epoch 87
Iter 96800: loss: 0.2876

[3.8425575e+01 4.7005563e+00 6.7439355e-02 6.3901976e-02 5.5045571e-02
 4.5110397e-02 4.1749593e-02 3.7656423e-02 3.1651452e-02 2.9796749e-02
 2.6703807e-02 2.5180154e-02 1.7762261e-02]
iter 88	train cost	0.28760531544685364	total variance to retain	1.3589859008789062e-05
Epoch 88
Iter 97900: loss: 0.2924

[3.7033035e+01 5.0383539e+00 6.7387320e-02 6.5423280e-02 5.3016804e-02
 4.2706329e-02 4.1185290e-02 3.6643524e-02 3.1660225e-02 2.8882442e-02
 2.5280286e-02 2.4115831e-02 2.2497622e-02]
iter 89	train cost	0.29242226481437683	total variance to retain	1.430511474609375e-05
Epoch 89
Iter 99000: loss: 0.2880

[3.9653172e+01 4.2228394e+00 1.0059174e-01 6.6314489e